In [27]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

import torch

In [22]:
# ToDo find a model that runs faster than  HuggingFaceH4/zephyr-7b-beta
model_name = "stabilityai/stablelm-zephyr-3b"

# If you have a GPU available you can use this code
# bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

# Instantiate one of the model classes of the library (with a causal language modeling head) from a pretrained model.
model = AutoModelForCausalLM.from_pretrained(model_name)
# Instantiate one of the tokenizer classes of the library from a pretrained model vocabulary.
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
# First, create a text_generation pipeline using the loaded model and its tokenizer.
# The pipeline() makes it simple to use any model from the Hub for inference on language, and other tasks.

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [31]:
# Create a prompt template
# this should follow the format of the model, so make sure to use the appropriate formatting.
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

# You can also use tokenizer.apply_chat_template to convert a list of messages (as dicts: {'role': 'user', 'content': '(...)'})
# into a string with the appropriate chat format.

In [32]:
# Finally, we need to combine the llm_chain with the retriever to create a RAG chain.
# We pass the original question through to the final generation step, as well as the retrieved context docs
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

db = FAISS.load_local(folder_path="faiss_db/", embeddings=embeddings, index_name="nasa_index", allow_dangerous_deserialization=True)

# This retriever returns the top 4 similar chunks
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

/Users/katie/Coding/NASA_RAG/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
question = "Tell me about auroras."

# Answer with just the model itself, no context added. Took too long to compute.
#llm_chain.invoke({"context": "", "question": question})

In [34]:
# A RAG response considering our NASA context
rag_chain.invoke(question)

"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content='NASA-Supported Team Discovers Aurora-Like Radio Bursts Above Sunspot \\n On Earth (and other planets such as Jupiter and Saturn), auroras shimmer in the night sky when solar particles are caught up in the planet’s magnetic field and get pulled toward the poles, where magnetic field lines converge. As they accelerate poleward, the particles generate intense radio emissions at frequencies around a few hundred kilohertz and then smash into atoms in the atmosphere, causing them to emit light as auroras.', metadata={'source': 'nasa_articles/nasa_article_536.md'}), Document(page_content='An Aurora in Another Light \\n NASA/Lauren Dauphin and Wanmei Liang; NOAA\\n\\nThe Visible Infrared Imaging Radiometer Suite sensor on the NOAA-NASA Suomi NPP satellite captured this image of the aurora borealis, or northern lights, over western Canada at 3:23 a.m. MST (5:23 a.m. EST) on 

RAG response considering context:
Aurora phenomena, commonly referred to as auroras, occur in the Earth's polar regions when charged particles from the Sun interact with the planet's magnetic field and get pulled towards the poles. These charged particles, primarily consisting of protons and electrons, get accelerated when they collide with the magnetic field, which causes them to radiate intense electromagnetic radiation at frequencies around a few hundred kilohertz. When these charged particles reach the Earth's upper atmosphere, they collide with atmospheric atoms, causing them to emit light, resulting in the beautiful auroral displays we observe in the night sky. \n\nThere are two main types of auroras: the Northern Lights (aurora borealis) and the Southern Lights (aurora australis). The Northern Lights are typically observed at higher latitudes, particularly around the Arctic Circle, while the Southern Lights are more commonly seen at lower latitudes, mainly in Australia, Antarctica, and the subpolar regions close to the equator. Both types of auroras result from the interaction between the Sun's charged particles and Earth's magnetic field. However, different factors, such as the strength of the solar activity and the orientation of the interplanetary magnetic field, may influence the type and intensity of the aurora.\n\nAuroras are not only visually stunning but also play an essential role in the Earth's radiation belts and space weather. They emit a significant amount of energy, contributing to the heating of the polar regions. Additionally, these radiations can interfere with communication systems and navigation equipment, making them important objects of study for researchers and scientists interested in understanding the complex interactions between the Earth's magnetic field, the Sun, and the atmosphere."